In [1]:
## Importing Required Libraries
import cv2
import numpy as np
from matplotlib import pyplot as plt

In [14]:
## Loading the image
img = cv2.imread("./images/toast_15.png")
img = cv2.resize(img, (200,250))

In [15]:
## Marking the center
img = cv2.circle(img,(100,125),1,(0,255,0),2)

In [16]:
## Canny edge filter

def auto_canny(image, sigma=0.33):
	v = np.median(image)
	lower = int(max(0, (1.0 - sigma) * v))
	upper = int(min(255, (1.0 + sigma) * v))
	edged = cv2.Canny(image, lower, upper)

	return edged

In [17]:
## Image preprocessing and contouring

gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
gray = cv2.GaussianBlur(gray, (5, 5), 0)
otsu_threshold, image_result = cv2.threshold(
    gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU,
)
edges = auto_canny(image_result,0.5)
cnts = cv2.findContours(edges.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

In [18]:
## Detecting the Circles and storing their centers in an array

lines = []
count = 0
detected_circles = cv2.HoughCircles(edges, 
                   cv2.HOUGH_GRADIENT, 1, 20, param1 = 45,
               param2 = 18, minRadius = 3, maxRadius = 25)

if detected_circles is not None:
  
    detected_circles = np.uint16(np.around(detected_circles))
  
    for pt in detected_circles[0, :]:
        a, b, r = pt[0], pt[1], pt[2]
  
        cv2.circle(img, (a, b), r, (0, 255, 0), 2)
        print(a,b)
        if(count == 7):
            break
        count +=1
        lines.append((pt[0],pt[1]))

        cv2.circle(img, (a, b), 1, (0, 0, 255), 3)
        cv2.imshow("Detected Circle", img)
        cv2.waitKey(0)
        cv2.destroyAllWindows()

56 76
136 74
116 106
94 76
80 110
116 48
80 48
66 186


In [19]:
## Arranging the points in ascending order

lines = np.array(lines)
points = lines.reshape((-1, 1, 2))
sortedArr = lines[lines[:,1].argsort()]
sortedArr

array([[116,  48],
       [ 80,  48],
       [136,  74],
       [ 56,  76],
       [ 94,  76],
       [116, 106],
       [ 80, 110]], dtype=uint16)

In [51]:
xt = sortedArr[0][0]
yt = sortedArr[0][1]
xt2 = sortedArr[1][0]
yt2 = sortedArr[1][1]
xb = sortedArr[5][0]
yb = sortedArr[5][1]
xb2 = sortedArr[6][0]
yb2 = sortedArr[6][1]
xm = sortedArr[2][0]
ym = sortedArr[2][1]
xm2 = sortedArr[4][0]
ym2 = sortedArr[4][1]
xc = sortedArr[3][0]
yc = sortedArr[3][1]

def distance(x1,x2,y1,y2):
    dist = np.sqrt(np.square(x2-x1) + np.square(y2-y1))
    return dist

In [55]:
def orientation(xt=0,yt=0,xt2=0,yt2=0,xb=0,yb=0,xb2=0,yb2=0,xm=0,ym=0,xm2=0,ym2=0,xc=0,yc=0):

    # From a reference image
    l0 = 37
    l02 = 86
    d0 = 56
    l1 = 0
    l2 = 0
    o1 = 0
    o2 = 0
    d1 = 0
    d2 = 0

    if xt != 0 and xt2 !=0:
        l1 = distance(xt,xt2,yt,yt2)

    if xb != 0 and xb2 !=0:
        l2 = distance(xb,xb2,yb,yb2)

    # For finding the orientation
    o1 = distance(xt,xm,yt,ym)
    o2 = distance(xt2,xm2,yt2,ym2)

    o3 = distance(xm,xb,ym,yb)
    o4 = distance(xm2,xb2,ym2,yb2)

    oleft = (o1+o3)/2
    oright = (o2+o4)/2

    if xt !=0 and xb !=0:
        d1 = distance(xt,xb,yt,yb)
    if xt2!= 0 and xb2!=0:
        d2 = distance(xt2,xb2,yt2,yb2)
        
    lf = (l1+l2)/2
    df = (d1+d2)/2
    
    ## For Finding the angle
    hor = np.arccos(lf/l0)
    vert = np.arccos(d0/df)

    if(oleft<oright):
        hor = -hor
    if(l1>l2):
        vert = -vert        

    return hor , vert

In [56]:
a , b = orientation(xt,yt,xt2,yt2,xb,yb,xb2,yb,xm,ym,xm2,ym2)
print(f"Horizontal rad:{a}, Vertical rad:{b}")
print(f"Horizontal = {a*(180/np.pi)}, Vertical = {b*(180/np.pi)}")

Horizontal rad:0.2330221256367575, Vertical rad:0.26337341611683224
Horizontal = 13.351184332153426, Vertical = 15.090185179437302


C:\Users\emaad\AppData\Local\Temp\ipykernel_19812\4072747429.py:17: RuntimeWarning: overflow encountered in ushort_scalars
  dist = np.sqrt(np.square(x2-x1) + np.square(y2-y1))


##### As 0bserved above we are able to get an angle of 13.3 degrees in horizontal direction which is 2.7 degrees less than the original degrees but we observe vast difference in Vertical angle as original vertical angle is 5 degrees.